The code below simulates hands in a card game of my own design that is currently named "Prism". The game is based off a simpler game known as "Goofspiel" or "GOPS" (Game of Pure Strategy).

The deck is made up of 8 suits:
* Red
* Orange
* Yellow
* Green
* Blue
* Indigo
* Violet
* Black
* White

The first suits 7 (The player suits) have 10 ranks, 1 through 10. Each rank has two copies. The black suit contains only 7 cards, all rank -10. The white suit contains 3 cards without any rank. This creates a total of 150 cards.

Without going into detail on how the game works, each player is attempting to build different hands from a limited selection of auction cards. The amount of available auction cards changes depending on how many players are participating. Using the Monte Carlo method we can determine the odds of various hands occuring (or more generally the odds of particular cards appearing) within the available auction cards based on the restrictions set in place for each hand type.

Those 8 hands include:
* Pairs
  - Two Cards of the same rank
* X of a kind
  - At least three cards of the same rank
* Runs
  - At least three cards with ranks in sequence
  - If a card in the sequence is also a pair, then this is considered a "double run". In other words, the run is counted twice. This concept can be carried further to a "triple run" where a card in the sequence is also three of a kind, a "double double run" where two cards in the sequence are also have a pair, etc.
  - For this simulation I have only accounted for the first type of "double runs" where only one card in the sequence has multiples.
* Flushes
  - At least four cards of the same suit
* Flush Run
  - At least three cards of the same suit and ranks in sequence
* Rainbow
  - All seven suits
* Half Rainbow
  - At least four of the seven suits
* Blackhole
  - All negative ranked cards available in the deck
* White Cards
  - White cards do not create any hands on their own. However, they can cancel the negative effects of a black card. In the absence of a black card they can double the total value of the players auction hand. Thus, it seems pertinent to track the likelihood of the card appearing in the available auction cards.

For this analysis we count an occurance as true if the necessary cards appear anywhere within the available auction cards. The position of the auction card in the draw cycle and the chance of winning that auction card are not considered as that is out of the scope of this project and involves game theory which I have no knowledge of.

The generated table was created after simulating each hand for each player amount 1 million times.

In [33]:
# Importing necessary libraries
import random
import copy
from collections import Counter
from functools import reduce
import pandas as pd
pd.set_option('display.max_rows', 100)
import numpy as np

In [35]:
# Adjusts number of active suits and total available auction cards based on the number of players
def player_adjustment(players):
  if players == 2:
    total_suits = 2
    auction_cards = 10
    return (total_suits, auction_cards)
  elif players == 3:
    total_suits = 3
    auction_cards = 10
    return (total_suits, auction_cards)
  elif players == 4:
    total_suits = 4
    auction_cards = 20
    return (total_suits, auction_cards)
  elif players == 5:
    total_suits = 5
    auction_cards = 20
    return (total_suits, auction_cards)
  elif players == 6:
    total_suits = 6
    auction_cards = 30
    return (total_suits, auction_cards)
  elif players == 7:
    total_suits = 7
    auction_cards = 30
    return (total_suits, auction_cards)

# Builds an organized deck based on number of players
# Note this is actually creating the auction pile, not the entire deck.
def build_deck(players):
  organized_deck = []
  adjustment = player_adjustment(players)[0]
  ranks = ['1', '2', '3', '4', '5', '6', '7', '8', '9', 'T']
  suits = ['R', 'O', 'Y', 'G', 'B', 'I', 'V']
  black_cards = ['NJ', 'NJ', 'NJ', 'NJ', 'NJ', 'NJ', 'NJ']
  white_cards = ['WJ', 'WJ', 'WJ']
  for rank in ranks:
    for suit in suits[0:adjustment]:
      organized_deck.append(rank+suit)
  for card in black_cards[0:adjustment]:
    organized_deck.append(card)
  for card in white_cards:
    organized_deck.append(card)
  return organized_deck

# Returns the rank of all auction cards
def get_ranks(deck, players):
  auction_cards = player_adjustment(players)[1]
  cards = deck[0:auction_cards]
  list_of_ranks = []
  for i in range(len(cards)):
    list_of_ranks.append(cards[i][0])
  return list_of_ranks

# Returns the suit of all auction cards
def get_suits(deck, players):
  auction_cards = player_adjustment(players)[1]
  cards = deck[0:auction_cards]
  list_of_ranks = []
  for i in range(len(cards)):
    list_of_ranks.append(cards[i][1])
  return list_of_ranks

# Returns the number of pairs in the available auction cards
def pair(deck, players):
  check_list = get_ranks(deck, players)
  results = sum(i//2 for i in Counter(check_list).values())
  return results

# Returns the number of three of a kind in the available auction cards
def three_kind(deck, players):
  check_list = get_ranks(deck, players)
  results = sum(i//3 for i in Counter(check_list).values())
  return results

# Returns the number of four of a kind in the available auction cards
def four_kind(deck, players):
  check_list = get_ranks(deck, players)
  results = sum(i//4 for i in Counter(check_list).values())
  return results

# Returns the number of five of a kind in the available auction cards
def five_kind(deck, players):
  check_list = get_ranks(deck, players)
  results = sum(i//5 for i in Counter(check_list).values())
  return results

# Returns the number of six of a kind in the available auction cards
def six_kind(deck, players):
  check_list = get_ranks(deck, players)
  results = sum(i//6 for i in Counter(check_list).values())
  return results

# Returns the number of seven of a kind in the available auction cards
def seven_kind(deck, players):
  check_list = get_ranks(deck, players)
  results = sum(i//7 for i in Counter(check_list).values())
  return results

# Returns the number off flushes in the available auction cards
def flush(deck, players):
  check_list = get_suits(deck, players)
  results = sum(i//4 for i in Counter(check_list).values())
  return results

# Checks whether a black hole occurs
def black_hole(deck, players):
  cards = get_ranks(deck, players)
  count_of_cards = Counter(cards)
  black_cards = count_of_cards['N']
  if black_cards == 3:
    return True

# Checks whether a rainbow occurs
def rainbow(deck, players):
  suit_counts = get_suit_counts(deck, players)
  R = suit_counts[0]
  O = suit_counts[1]
  Y = suit_counts[2]
  G = suit_counts[3]
  B = suit_counts[4]
  I = suit_counts[5]
  V = suit_counts[6]
  if R > 0 and O > 0 and Y > 0 and G > 0 and B > 0 and I > 0 and V > 0:
    return True

def get_suit_counts(deck, players):
  cards = get_suits(deck, players)
  count_of_cards = Counter(cards)
  R = count_of_cards['R']
  O = count_of_cards['O']
  Y = count_of_cards['Y']
  G = count_of_cards['G']
  B = count_of_cards['B']
  I = count_of_cards['I']
  V = count_of_cards['V']
  suit_counts = [R, O, Y, G, B, I, V]
  return suit_counts

# Checks whether a half rainbow occurs
def half_rainbow(deck, players):
  suit_counts = get_suit_counts(deck, players)
  r = suit_counts[0]
  o = suit_counts[1]
  y = suit_counts[2]
  g = suit_counts[3]
  b = suit_counts[4]
  i = suit_counts[5]
  v = suit_counts[6]
  suits_present = 0
  if r > 0: suits_present += 1
  if o > 0: suits_present += 1
  if y > 0: suits_present += 1
  if g > 0: suits_present += 1
  if b > 0: suits_present += 1
  if i > 0: suits_present += 1
  if v > 0: suits_present += 1
  if suits_present >= 4 and suits_present < 7:
    return True

# Function that converts the string ranks to integers
def ranks_to_integers(rank):
  match rank:
    case '1': return 1
    case '2': return 2
    case '3': return 3
    case '4': return 4
    case '5': return 5
    case '6': return 6
    case '7': return 7
    case '8': return 8
    case '9': return 9
    case 'T': return 10
    case 'N': return -10
    case 'W': return 20

# Checks whether the values in a list are sequential
def is_sequential(list_of_values):
  return np.sum (np.diff(sorted(list_of_values))) == len(list_of_values) - 1

# Returns the number of runs present in the available auction cards
def run(deck, players, run_length):
  cards = get_ranks(deck, players)
  converted_rank = []
  run_counter = 0
  # Creating set of ranks converted to integers
  for card in cards:
    converted = ranks_to_integers(card)
    converted_rank.append(converted)
  converted_rank_set = list(set(converted_rank))
  list_of_sequences = []
  # Creating list of card sequences to check
  if run_length <= len(converted_rank_set):
    for i in range(len(converted_rank_set) - (run_length - 1)):
      sequences_to_check = converted_rank_set[i:i+run_length]
      list_of_sequences.append(sequences_to_check)
  # Checking whether each sequence is sequential
  for sequence in list_of_sequences:
    if is_sequential(sequence):
      run_counter += 1
  return run_counter

# Returns the number of flush_runs present in the available auction cards
def flush_run(deck, players, run_length):
  # Counting the number of occurences of each suit
  cards = deck[0:player_adjustment(players)[1]]
  check_list = get_suits(deck, players)
  count_of_cards = Counter(check_list)
  R = count_of_cards['R']
  O = count_of_cards['O']
  Y = count_of_cards['Y']
  G = count_of_cards['G']
  B = count_of_cards['B']
  I = count_of_cards['I']
  V = count_of_cards['V']
  count_of_suits = [('R', R), ('O', O), ('Y', Y), ('G', G), ('B', B), ('I', I), ('V', V)]
  # Creating list of suits that have flush potential
  valid_suits = []
  for i in range(len(count_of_suits)):
    if count_of_suits[i][1] >= 3: valid_suits.append(count_of_suits[i][0])
  # Creating list of ranks partitioned by suit
  r_ranks = []
  o_ranks = []
  y_ranks = []
  g_ranks = []
  b_ranks = []
  i_ranks = []
  v_ranks = []
  for i in range(len(cards)):
    if cards[i][1] in valid_suits:
      match cards[i][1]:
        case 'R': r_ranks.append(cards[i][0])
        case 'O': o_ranks.append(cards[i][0])
        case 'Y': o_ranks.append(cards[i][0])
        case 'G': o_ranks.append(cards[i][0])
        case 'B': o_ranks.append(cards[i][0])
        case 'I': o_ranks.append(cards[i][0])
        case 'V': o_ranks.append(cards[i][0])
  suit_ranks = [r_ranks, o_ranks, y_ranks, g_ranks, b_ranks, i_ranks, v_ranks]
  # Creating list of ranks converted to integers
  converted_suit_rank = []
  for i in suit_ranks:
    if len(i) > 0:
      converted_rank = []
      for j in i:
        converted = ranks_to_integers(j)
        converted_rank.append(converted)
      converted_suit_rank.append(converted_rank)
  # Removing duplicates and sorting the converted ranks
  converted_suit_rank_set = []
  for i in converted_suit_rank:
    list_to_set = list(set(i))
    converted_suit_rank_set.append(list_to_set)
  # Creating list of sequences to check
  list_of_sequences = []
  for i in converted_suit_rank_set:
    if run_length <= len(i):
      for j in range(len(i) - (run_length - 1)):
        sequences_to_check = i[j:j+run_length]
        list_of_sequences.append(sequences_to_check)
  run_counter = 0
  # Checking whether each sequence is sequential
  for sequence in list_of_sequences:
    if is_sequential(sequence):
      run_counter += 1
  return run_counter

# Returns list of the number of runs present in the available auction cards
def multi_run(deck, players, run_length):
  # Obtaining the ranks and converting them to integers
  ranks = get_ranks(deck, players)
  converted_rank = []
  for rank in ranks:
    converted = ranks_to_integers(rank)
    converted_rank.append(converted)
  converted_rank_set = list(set(converted_rank))

  # Creating list of pairs, triples, and so on
  dict_of_converted_ranks = dict(Counter(converted_rank))
  card_pair = []
  card_triple = []
  card_quadruple = []
  card_quintuple = []
  card_sextuple = []
  card_septuple = []
  list_of_converted_ranks = [(k, v) for k,v in dict_of_converted_ranks.items()]
  for i in range(len(list_of_converted_ranks)):
    value = list_of_converted_ranks[i][1]
    card = list_of_converted_ranks[i][0]
    if value//2: card_pair.append(card)
    if value//3: card_triple.append(card)
    if value//4: card_quadruple.append(card)
    if value//5: card_quintuple.append(card)
    if value//6: card_sextuple.append(card)
    if value//7: card_septuple.append(card)

  # Creating list of card sequences to check
  list_of_sequences = []
  if run_length <= len(converted_rank_set):
    for i in range(len(converted_rank_set) - (run_length - 1)):
      sequences_to_check = converted_rank_set[i:i+run_length]
      list_of_sequences.append(sequences_to_check)

  # Counting occurances of each multiple run type
  double_run_counter = 0
  triple_run_counter = 0
  quadruple_run_counter = 0
  quintuple_run_counter = 0
  sextuple_run_counter = 0
  septuple_run_counter = 0
  for sequence in list_of_sequences:
    for i in card_pair:
      if is_sequential(sequence) and i in sequence:
        double_run_counter += 1
    for i in card_triple:
      if is_sequential(sequence) and i in sequence:
        triple_run_counter += 1
    for i in card_quadruple:
      if is_sequential(sequence) and i in sequence:
        quadruple_run_counter += 1
    for i in card_quintuple:
      if is_sequential(sequence) and i in sequence:
        quintuple_run_counter += 1
    for i in card_sextuple:
      if is_sequential(sequence) and i in sequence:
        sextuple_run_counter += 1
    for i in card_septuple:
      if is_sequential(sequence) and i in sequence:
        septuple_run_counter += 1
  count_of_multi_runs = [double_run_counter, triple_run_counter,
                         quadruple_run_counter, quintuple_run_counter,
                         sextuple_run_counter, septuple_run_counter]
  return count_of_multi_runs

# Returns list of counts of each run type
def run_counter(deck, players):
  count_of_runs = []
  for i in range(3,11):
    results = run(deck, players, i)
    count_of_runs.append(results)
  return count_of_runs

# Returns list of counts for each flush run type
def flush_run_counter(deck, players):
  count_of_flushes = []
  for i in range(3,11):
    results = flush_run(deck, players, i)
    count_of_flushes.append(results)
  return count_of_flushes

# Returns list of lists for each run length and multi runs it contains
def multi_run_counter(deck, players):
  count_of_multi_runs = []
  for i in range(3,11):
    results = multi_run(deck, players, i)
    count_of_multi_runs.append(results)
  return count_of_multi_runs

def white_card(deck, players):
  ranks = get_ranks(deck, players)
  if 'W' in ranks: return True

# Randomly shuffles the created deck and checks to see if given conditions are met
# Returns dataframe of the percentage that a positive result occurred
# The more iterations the more auccurate the results
def MonteCarlo(iterations, players):

  # building the deck
  # hard setting the players due to a change in the rule set
  # originally the deck size changed based on the number of players
  # This is no longer the case
  organized_deck = build_deck(7)

  # Initiate the hand counts
  final_no_pair_count = 0
  final_pair_count = 0
  final_two_pair_count = 0
  final_three_pair_count = 0
  final_four_pair_count = 0
  final_five_pair_count = 0
  final_six_pair_count = 0
  final_seven_pair_count = 0
  final_eight_pair_count = 0
  final_nine_pair_count = 0
  final_ten_pair_count = 0
  final_eleven_pair_count = 0
  final_twelve_pair_count = 0
  final_thirteen_pair_count = 0
  final_fourteen_pair_count = 0
  final_fifteen_pair_count = 0
  final_three_kind_count = 0
  final_four_kind_count = 0
  final_five_kind_count = 0
  final_six_kind_count = 0
  final_seven_kind_count = 0
  final_flush_count = 0
  final_run_3_count = 0
  final_run_4_count = 0
  final_run_5_count = 0
  final_run_6_count = 0
  final_run_7_count = 0
  final_run_8_count = 0
  final_run_9_count = 0
  final_run_10_count = 0
  final_flush_run_3_count = 0
  final_flush_run_4_count = 0
  final_flush_run_5_count = 0
  final_flush_run_6_count = 0
  final_flush_run_7_count = 0
  final_flush_run_8_count = 0
  final_flush_run_9_count = 0
  final_flush_run_10_count = 0
  final_double_run_of_3_count = 0
  final_triple_run_of_3_count = 0
  final_quadruple_run_of_3_count = 0
  final_quintuple_run_of_3_count = 0
  final_sextuple_run_of_3_count = 0
  final_septuple_run_of_3_count = 0
  final_double_run_of_4_count = 0
  final_triple_run_of_4_count = 0
  final_quadruple_run_of_4_count = 0
  final_quintuple_run_of_4_count = 0
  final_sextuple_run_of_4_count = 0
  final_septuple_run_of_4_count = 0
  final_double_run_of_5_count = 0
  final_triple_run_of_5_count = 0
  final_quadruple_run_of_5_count = 0
  final_quintuple_run_of_5_count = 0
  final_sextuple_run_of_5_count = 0
  final_double_run_of_6_count = 0
  final_triple_run_of_6_count = 0
  final_quadruple_run_of_6_count = 0
  final_quintuple_run_of_6_count = 0
  final_double_run_of_7_count = 0
  final_triple_run_of_7_count = 0
  final_quadruple_run_of_7_count = 0
  final_double_run_of_8_count = 0
  final_triple_run_of_8_count = 0
  final_double_run_of_9_count = 0
  black_hole_counter = 0
  rainbow_counter = 0
  half_rainbow_counter = 0
  final_flush_count = 0
  white_card_counter = 0

  # Beginning the loop
  for i in range(iterations):

    # Copy the deck and shuffle it
    deck = copy.deepcopy(organized_deck)
    random.shuffle(deck)

    # Counting pairs
    pair_count = pair(deck, players)
    match pair_count:
      case 0: final_no_pair_count +=1
      case 1: final_pair_count +=1
      case 2: final_two_pair_count +=1
      case 3: final_three_pair_count +=1
      case 4: final_four_pair_count +=1
      case 5: final_five_pair_count +=1
      case 6: final_six_pair_count +=1
      case 7: final_seven_pair_count +=1
      case 8: final_eight_pair_count +=1
      case 9: final_nine_pair_count +=1
      case 10: final_ten_pair_count +=1
      case 11: final_eleven_pair_count +=1
      case 12: final_twelve_pair_count +=1
      case 13: final_thirteen_pair_count +=1
      case 14: final_fourteen_pair_count +=1
      case 15: final_fifteen_pair_count +=1

    # Counting Flushes
    flush_count = flush(deck, players)
    final_flush_count += flush_count

    # Counting x of a kind
    three_kind_count = three_kind(deck, players)
    final_three_kind_count += three_kind_count
    four_kind_count = four_kind(deck, players)
    final_four_kind_count += four_kind_count
    five_kind_count = five_kind(deck, players)
    final_five_kind_count += five_kind_count
    six_kind_count = six_kind(deck, players)
    final_six_kind_count += six_kind_count
    seven_kind_count = seven_kind(deck, players)
    final_seven_kind_count += seven_kind_count

    # Counting flushes
    flush_count = flush(deck, players)
    final_flush_count += flush_count

    # Counting runs
    run_count = run_counter(deck, players) # Returns list
    run_3 = run_count[0]
    final_run_3_count += run_3
    run_4 = run_count[1]
    final_run_4_count += run_4
    run_5 = run_count[2]
    final_run_5_count += run_5
    run_6 = run_count[3]
    final_run_6_count += run_6
    run_7 = run_count[4]
    final_run_7_count += run_7
    run_8 = run_count[5]
    final_run_8_count += run_8
    run_9 = run_count[6]
    final_run_9_count += run_9
    run_10 = run_count[7]
    final_run_10_count += run_10

    # Counting flush runs
    flush_run_count = flush_run_counter(deck, players) # Returns list
    flush_run_3 = flush_run_count[0]
    final_flush_run_3_count += flush_run_3
    flush_run_4 = flush_run_count[1]
    final_flush_run_4_count += flush_run_4
    flush_run_5 = flush_run_count[2]
    final_flush_run_5_count += flush_run_5
    flush_run_6 = flush_run_count[3]
    final_flush_run_6_count += flush_run_6
    flush_run_7 = flush_run_count[4]
    final_flush_run_7_count += flush_run_7
    flush_run_8 = flush_run_count[5]
    final_flush_run_8_count += flush_run_8
    flush_run_9 = flush_run_count[6]
    final_flush_run_9_count += flush_run_9
    flush_run_10 = flush_run_count[7]
    final_flush_run_10_count += flush_run_10

    # Counting Multi Runs
    multi_run_count = multi_run_counter(deck, players) # Returns list of lists
    double_run_of_3 = multi_run_count[0][0]
    final_double_run_of_3_count += double_run_of_3
    triple_run_of_3 = multi_run_count[0][1]
    final_triple_run_of_3_count += triple_run_of_3
    quadruple_run_of_3 = multi_run_count[0][2]
    final_quadruple_run_of_3_count += quadruple_run_of_3
    quintuple_run_of_3 = multi_run_count[0][3]
    final_quintuple_run_of_3_count += quintuple_run_of_3
    sextuple_run_of_3 = multi_run_count[0][4]
    final_sextuple_run_of_3_count += sextuple_run_of_3
    septuple_run_of_3 = multi_run_count[0][5]
    final_septuple_run_of_3_count += septuple_run_of_3

    double_run_of_4 = multi_run_count[1][0]
    final_double_run_of_4_count += double_run_of_4
    triple_run_of_4 = multi_run_count[1][1]
    final_triple_run_of_4_count += triple_run_of_4
    quadruple_run_of_4 = multi_run_count[1][2]
    final_quadruple_run_of_4_count += quadruple_run_of_4
    quintuple_run_of_4 = multi_run_count[1][3]
    final_quintuple_run_of_4_count += quintuple_run_of_4
    sextuple_run_of_4 = multi_run_count[1][4]
    final_sextuple_run_of_4_count += sextuple_run_of_4
    septuple_run_of_4 = multi_run_count[1][5]
    final_septuple_run_of_4_count += septuple_run_of_4

    double_run_of_5 = multi_run_count[2][0]
    final_double_run_of_5_count += double_run_of_5
    triple_run_of_5 = multi_run_count[2][1]
    final_triple_run_of_5_count += triple_run_of_5
    quadruple_run_of_5 = multi_run_count[2][2]
    final_quadruple_run_of_5_count += quadruple_run_of_5
    quintuple_run_of_5 = multi_run_count[2][3]
    final_quintuple_run_of_5_count += quintuple_run_of_5
    sextuple_run_of_5 = multi_run_count[2][4]
    final_sextuple_run_of_5_count += sextuple_run_of_5

    double_run_of_6 = multi_run_count[3][0]
    final_double_run_of_6_count += double_run_of_6
    triple_run_of_6 = multi_run_count[3][1]
    final_triple_run_of_6_count += triple_run_of_6
    quadruple_run_of_6 = multi_run_count[3][2]
    final_quadruple_run_of_6_count += quadruple_run_of_6
    quintuple_run_of_6 = multi_run_count[3][3]
    final_quintuple_run_of_6_count += quintuple_run_of_6

    double_run_of_7 = multi_run_count[4][0]
    final_double_run_of_7_count += double_run_of_7
    triple_run_of_7 = multi_run_count[4][1]
    final_triple_run_of_7_count += triple_run_of_7
    quadruple_run_of_7 = multi_run_count[4][2]
    final_quadruple_run_of_7_count += quadruple_run_of_7

    double_run_of_8 = multi_run_count[5][0]
    final_double_run_of_8_count += double_run_of_8
    triple_run_of_8 = multi_run_count[5][1]
    final_triple_run_of_8_count += triple_run_of_8

    double_run_of_9 = multi_run_count[6][0]
    final_double_run_of_9_count += double_run_of_9

    # Counting black holes, rainbows, and white cards
    if black_hole(deck, players): black_hole_counter += 1
    if rainbow(deck, players): rainbow_counter += 1
    if half_rainbow(deck, players): half_rainbow_counter += 1
    if white_card(deck, players): white_card_counter += 1

  # Creating list of final results
  final_counts = [final_no_pair_count, final_pair_count
                  ,final_two_pair_count, final_three_pair_count
                  ,final_four_pair_count, final_five_pair_count
                  ,final_six_pair_count, final_seven_pair_count
                  ,final_eight_pair_count, final_nine_pair_count
                  ,final_ten_pair_count, final_eleven_pair_count
                  ,final_twelve_pair_count, final_thirteen_pair_count
                  ,final_fourteen_pair_count, final_fifteen_pair_count
                  ,final_three_kind_count, final_four_kind_count
                  ,final_five_kind_count, final_six_kind_count
                  ,final_seven_kind_count, final_flush_count, final_run_3_count
                  ,final_run_4_count, final_run_5_count, final_run_6_count
                  ,final_run_6_count, final_run_7_count, final_run_8_count
                  ,final_run_9_count, final_run_10_count
                  ,final_flush_run_3_count, final_flush_run_4_count
                  ,final_flush_run_5_count, final_flush_run_6_count
                  ,final_flush_run_8_count, final_flush_run_9_count
                  ,final_flush_run_10_count, final_double_run_of_3_count
                  ,final_triple_run_of_3_count, final_quadruple_run_of_3_count
                  ,final_quintuple_run_of_3_count, final_sextuple_run_of_3_count
                  ,final_septuple_run_of_3_count, final_double_run_of_4_count
                  ,final_triple_run_of_4_count, final_quadruple_run_of_4_count
                  ,final_quintuple_run_of_4_count, final_sextuple_run_of_4_count
                  ,final_septuple_run_of_4_count, final_double_run_of_5_count
                  ,final_triple_run_of_5_count, final_quadruple_run_of_5_count
                  ,final_quintuple_run_of_5_count, final_sextuple_run_of_5_count
                  ,final_double_run_of_6_count, final_triple_run_of_6_count
                  ,final_quadruple_run_of_6_count, final_quintuple_run_of_6_count
                  ,final_double_run_of_7_count, final_triple_run_of_7_count
                  ,final_quadruple_run_of_7_count, final_double_run_of_8_count
                  ,final_triple_run_of_8_count, final_double_run_of_9_count
                  ,black_hole_counter, rainbow_counter, half_rainbow_counter, white_card_counter]

  # Calculating the probablility for each hand
  final_aggregations = []
  for i in final_counts:
    percentage = ((i/iterations)*100)
    if percentage > 100:
      percentage = 100.0
    final_aggregations.append(percentage)

  # Creating the data frame
  data = {
    'Hand_Type': ['No_Pair', 'Pair','Two_Pair','Three_Pair','Four_Pair'
                  ,'Five_Pair','Six_Pair','Seven_Pair','Eight_Pair','Nine_Pair'
                  ,'Ten_Pair','Eleven_Pair','Twelve_Pair','Thirteen_Pair'
                  ,'Fourteen_Pair','Fifteen_Pair', 'Three_Kind', 'Four_Kind'
                  ,'Five_Kind', 'Six_Kind','Seven_Kind', 'Flush', 'Run_of_3'
                  ,'Run_of_4', 'Run_of_5', 'Run_of_6','Run_of_7', 'Run_of_8'
                  ,'Run_of_9','Run_of_10', 'Flush_Run_3', 'Flush_Run_4'
                  ,'Flush_Run_5','Flush_Run_6','Flush_Run_7', 'Flush_Run_8'
                  ,'Flush_Run_9', 'Flush_Run_10', 'Double_Run_of_3'
                  ,'Triple_Run_of_3', 'Quadruple_Run_of_3'
                  ,'Quintuple_Run_of_3', 'Sextuple_Run_of_3', 'Septuple_Run_of_3'
                  ,'Double_Run_of_4', 'Triple_Run_of_4', 'Quadruple_Run_of_4'
                  ,'Quintuple_Run_of_4', 'Sextuple_Run_of_4', 'Septuple_Run_of_4'
                  ,'Double_Run_of_5', 'Triple_Run_of_5', 'Quadruple_Run_of_5'
                  ,'Quintuple_Run_of_5', 'Sextuple_Run_of_5'
                  ,'Double_Run_of_6', 'Triple_Run_of_6', 'Quadruple_Run_of_6'
                  ,'Quintuple_Run_of_6', 'Double_Run_of_7'
                  ,'Triple_Run_of_7', 'Quadruple_Run_of_7', 'Double_Run_of_8'
                  ,'Triple_Run_of_8', 'Double_Run_of_9','Black_Hole','Rainbow'
                  ,'Half_Rainbow', 'White_Card']
    ,'Percentage': final_aggregations
        }
  df = pd.DataFrame(data)
  return df

In [36]:
# Running one million hand simulations for each player count
data_frames = []
for i in range(2,8):
  iterations = 1000000
  df = MonteCarlo(iterations, i)
  players = str(i)
  players_column = players + '_Players'
  df.rename(columns={'Hand_Type': 'Hand_Type', 'Percentage': players_column}, inplace = True)
  data_frames.append(df)

# merging the list of data frames
df_all = reduce(lambda  left,right: pd.merge(left,right,on=['Hand_Type'],how='outer'), data_frames)
df_all

,Hand_Type,2_Players,3_Players,4_Players,5_Players,6_Players,7_Players
0,No_Pair,0.5942,0.5722,0.0000,0.0000,0.0000,0.0000
1,Pair,12.0468,12.0632,0.0000,0.0000,0.0000,0.0000
2,Two_Pair,42.6585,42.6894,0.0000,0.0000,0.0000,0.0000
3,Three_Pair,37.2075,37.2000,0.0000,0.0000,0.0000,0.0000
4,Four_Pair,7.3433,7.3183,0.0444,0.0435,0.0000,0.0000
5,Five_Pair,0.1497,0.1569,2.9159,2.9170,0.0000,0.0000
6,Six_Pair,0.0000,0.0000,23.0469,23.1059,0.0000,0.0000
7,Seven_Pair,0.0000,0.0000,45.1121,45.0948,0.0000,0.0000
8,Eight_Pair,0.0000,0.0000,25.3138,25.2938,0.0000,0.0000
9,Nine_Pair,0.0000,0.0000,3.5093,3.4837,0.0460,0.0449


In [38]:
df_all.to_excel('/content/drive/MyDrive/Prism/Hand Odds.xlsx', index=False)